In [ ]:
from dglchem.utils import featurizer as fe
from dglchem.utils import featurizer_old as fo
from rdkit import Chem

mol = Chem.MolFromSmiles('COO')
feat = fe.AtomFeaturizer()
feat_old = fo.AtomFeaturizerOld()

bond = fe.BondFeaturizer()
bond_old = fo.BondFeaturizer()

assert (feat(mol)[0] == feat_old(mol)['h'][0]).all()
assert (bond(mol)[0] == bond_old(mol)['e'][0]).all()
print('Featurizers are fine!')

%timeit feat(mol)
%timeit feat_old(mol)
%timeit bond(mol)
%timeit bond_old(mol)

Featurizers are fine!
174 µs ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
81.2 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
33.9 µs ± 68.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [1]:
from dglchem.data import BradleyDoublePlus
data = BradleyDoublePlus()